In [ ]:
# ========================================================================
#                               FINANCIAL OCR 
#                     *** Proof of Concept Testing ***
# ========================================================================
#
# What this script does:
# 1. Lets you UPLOAD a financial PDF.
# 2. Converts each PDF page into an image.
# 3. Uses Tesseract OCR to extract:
#       - Narrative text (directors' report, notes, etc.)
#       - Table-like numeric rows (P&L, balance sheet, notes tables).
# 4. Builds:
#       - A "Narrative" table (by sentence).
#       - A "Tables" table (by line item / year).
# 5. Calculates key metrics:
#       - Turnover, Profit, Cash, Net assets, etc.
# 6. Tries to compute year-on-year % changes (if 2+ years exist).
# 7. Prints a clean human-readable SUMMARY in the Colab output.
# 8. Builds Turnover & Profit charts over the years (if data exists).
# 9. Exports:
#       - An Excel file (Narrative, Tables, Summary).
#       - A styled PDF report with:
#             * Key metrics table
#             * Directors' report highlights
#             * Notes highlights
#             * Charts embedded
#             * Sample of extracted table rows
#
# Requirements:
#   - Run in Google Colab.
#   - Upload a Financial PDF.
#
# ========================================================================


# ------------------------------------------------------------------------
# PART 1 — INSTALL SYSTEM & PYTHON DEPENDENCIES (COLAB-SPECIFIC)
# ------------------------------------------------------------------------
# These shell commands (starting with "!") are specific to Colab / Jupyter.
# They install system packages and Python libraries needed for the pipeline.

!apt update -qq
!apt install -y -qq tesseract-ocr poppler-utils
!pip install -q pytesseract pdf2image regex nltk openpyxl reportlab matplotlib


# ------------------------------------------------------------------------
# PART 2 — IMPORT PYTHON LIBRARIES
# ------------------------------------------------------------------------
# Now import all the Python modules we’ll use.

import pytesseract                         # Python wrapper around Tesseract OCR
from pdf2image import convert_from_path    # Convert PDF pages to images
from pytesseract import Output             # Structured OCR output (dataframes)

from google.colab import files             # For uploading & downloading files

import pandas as pd                        # Dataframes & Excel export
import numpy as np                         # Numerical utilities
import regex as re                         # Advanced regular expressions

import nltk                                # Natural Language Toolkit
from nltk.tokenize import sent_tokenize    # Sentence splitting

import matplotlib.pyplot as plt            # Charts & graphs
from io import BytesIO                     # In-memory binary streams (for charts)

# ReportLab for building a PDF report
from reportlab.lib.pagesizes import A4
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer,
    Table as RLTable, TableStyle,
    Image as RLImage, PageBreak
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch

# Download NLTK sentence tokenizer data once in the Colab environment
nltk.download("punkt")


# ------------------------------------------------------------------------
# PART 3 — HELPER REGEX PATTERNS & EXTRACTION FUNCTIONS
# ------------------------------------------------------------------------
# These are small utilities that will be reuse throughout the script to
# extract numbers, years, dates, names, etc.


# Pattern to match money-like tokens, e.g.:
#   "€10,000", "£500", "1,250,300", "(35,000)"
_money_re = re.compile(r"(?:€|£)?\s*[-(]?\d[\d,\,\.]*\)?")

# Pattern to match years such as 2019, 2020, 2023
_year_re = re.compile(r"\b(20\d{2})\b")

# Pattern to match long-form dates (e.g. "12 March 2024")
_date_re = re.compile(
    r"\b\d{1,2}\s+"
    r"(January|February|March|April|May|June|July|August|September|October|November|December)"
    r"\s+\d{4}\b",
    flags=re.IGNORECASE
)

# Pattern to detect simple personal names (first + optional middle + last)
_name_re = re.compile(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+){0,2}\b")


def extract_financial_numbers(text):
    """
    Extracts all values that look like monetary or numeric amounts from a line of text.

    Example:
        "Turnover 4,863,792 3,965,645"
    might yield:
        [("4,863,792", 4863792.0), ("3,965,645", 3965645.0)]

    Also handles parentheses for negatives:
        "(35,000)" -> -35000.0
    """
    numbers = []

    for m in re.finditer(_money_re, text):
        original = m.group(0)

        # Convert "(1000)" to "-1000"
        cleaned = original.replace("(", "-").replace(")", "")

        # Keep only digits, decimal points, and minus sign
        numeric = re.sub(r"[^\d\.\-]", "", cleaned)

        # Filter out nonsense like "" or just "-"
        if numeric in ("", "-", ".", "-."):
            continue

        try:
            num = float(numeric.replace(",", ""))
            numbers.append((original.strip(), num))
        except:
            # If conversion fails, ignore this match
            pass

    return numbers


def extract_years(text):
    """Return a list of all years like 2020, 2021 found in text."""
    return _year_re.findall(text)


def extract_dates(text):
    """Return a list of long-form dates (day + month + year)."""
    return _date_re.findall(text)


def extract_names(text):
    """
    Roughly extract names of people from text, excluding obvious false positives
    like 'Company', 'Limited', 'Page', etc.
    """
    names = _name_re.findall(text)
    return [n for n in names if n.lower() not in ("company", "limited", "notes", "page")]


def sanitize_label(label):
    """Normalize whitespace in labels, removing extra spaces."""
    return re.sub(r"\s+", " ", label).strip()


# ------------------------------------------------------------------------
# SECTION CLASSIFIER — to categorise narrative sentences
# ------------------------------------------------------------------------
# Want to mark each sentence as belonging to:
#   - directors_report
#   - results
#   - assets
#   - liabilities
#   - equity
#   - notes
#   - accounting_records
#   - general
#
# This helps build a structured summary later (e.g., "Directors' Report Highlights").

SECTION_KEYWORDS = {
    "directors_report": [
        "director", "director's report", "directors report", "board"
    ],
    "results": [
        "profit", "loss", "turnover", "tax", "profit for the financial year"
    ],
    "assets": [
        "assets", "tangible", "stocks", "debtors", "inventory", "inventories"
    ],
    "liabilities": [
        "liabilities", "creditors"
    ],
    "equity": [
        "equity", "share capital", "reserves"
    ],
    "notes": [
        "notes to the financial statements", "notes to"
    ],
    "accounting_records": [
        "accounting records"
    ],
    "general": []
}


def classify_section(text):
    """
    Determine which section a piece of text likely belongs to by checking
    for certain keywords. If none match, return 'general'.
    """
    t = text.lower()
    for section, words in SECTION_KEYWORDS.items():
        for w in words:
            if w in t:
                return section
    return "general"


# ------------------------------------------------------------------------
# UPLOAD THE PDF (USER INTERACTION)
# ------------------------------------------------------------------------
# This shows a file chooser in Colab. Must select a financial PDF from
# your local computer. The rest of the script uses that PDF for processing.

print("Please upload your financial document (PDF):")
uploaded = files.upload()

if not uploaded:
    raise SystemExit("No file uploaded. Please run the cell again and upload a PDF.")

pdf_path = list(uploaded.keys())[0]
print("Using file:", pdf_path)


# ------------------------------------------------------------------------
# PART 4 — CONVERT PDF PAGES TO IMAGES
# ------------------------------------------------------------------------
# Tesseract OCR cannot read PDFs directly; it works on images.
# pdf2image.convert_from_path() creates an image (PIL Image) for each page.

print("\nConverting PDF pages to images (300 DPI)...")
pages = convert_from_path(pdf_path, dpi=300)
print(f"Conversion complete: {len(pages)} page(s) converted.")


# ------------------------------------------------------------------------
# PART 5 — OCR EACH PAGE (FULL TEXT + LINE-LEVEL DATA)
# ------------------------------------------------------------------------
# Perform OCR in two ways:
#   1) get full text for narrative analysis
#   2) get line-level bounding boxes and text for table-like structure parsing

print("\nRunning OCR on all pages...")

page_texts = []         # full text per page (for narrative)
all_lines_by_page = []  # line-level OCR with coordinates for each page

for page_index, page in enumerate(pages, start=1):
    # 1) Full-page text
    text = pytesseract.image_to_string(page)
    page_texts.append(text)

    # 2) Line-level OCR with bounding boxes
    df = pytesseract.image_to_data(page, output_type=Output.DATAFRAME)

    if 'text' not in df.columns:
        # If OCR completely failed on this page
        all_lines_by_page.append([])
        continue

    # Remove rows where 'text' is NaN
    df = df.dropna(subset=["text"])

    # Group by Tesseract's (block, paragraph, line) IDs to get one row per line
    grouped = df.groupby(["block_num", "par_num", "line_num"])
    lines = []

    for _, g in grouped:
        line_text = " ".join(g["text"].astype(str).tolist()).strip()
        if not line_text:
            continue

        left = int(g["left"].min())
        top = int(g["top"].min())
        right = int((g["left"] + g["width"]).max())
        width = right - left
        height = int((g["top"] + g["height"]).max() - top)

        lines.append((line_text, left, top, width, height))

    # Sort lines by vertical position then left-right, to approximate reading order
    lines = sorted(lines, key=lambda x: (x[2], x[1]))

    # Basic label-value merge:
    # If a line has no numbers but the next one does, merge them so we get
    # "Label  123,456  234,567" instead of "Label" and then numbers on the next line.
    merged_lines = []
    i = 0
    while i < len(lines):
        text_line, l, t, w, h = lines[i]
        nums = extract_financial_numbers(text_line)

        if len(nums) == 0 and i + 1 < len(lines):
            nxt_text, nl, nt, nw, nh = lines[i + 1]
            if extract_financial_numbers(nxt_text):
                merged_lines.append((
                    f"{text_line} {nxt_text}",
                    min(l, nl),
                    min(t, nt),
                    max(w, nw),
                    max(h, nh)
                ))
                i += 2
                continue

        merged_lines.append((text_line, l, t, w, h))
        i += 1

    all_lines_by_page.append(merged_lines)

print("OCR complete. Collected full text and line-level data.")


# ------------------------------------------------------------------------
# PART 6 — NARRATIVE SENTENCE EXTRACTION
# ------------------------------------------------------------------------
# Work with the full text (all pages combined). We'll:
#   1) Split it into sentences with NLTK.
#   2) For each sentence:
#        * detect which section it's from
#        * extract numeric values (current year, prior year where possible)
#        * extract names and dates
#   3) Store everything in a DataFrame df_narrative.

print("\nExtracting narrative sentences from OCR text...")

full_text = "\n".join(page_texts)
sentences = sent_tokenize(full_text)

narrative_rows = []

for s in sentences:
    s_clean = s.replace("\x0c", " ").strip()  # remove strange page-break characters
    if not s_clean:
        continue

    section = classify_section(s_clean)
    nums = extract_financial_numbers(s_clean)

    current_val = nums[0][1] if len(nums) > 0 else ""
    prior_val   = nums[1][1] if len(nums) > 1 else ""

    narrative_rows.append({
        "section":       section,
        "text":          s_clean,
        "value_current": current_val,
        "value_prior":   prior_val,
        "names":         ", ".join(extract_names(s_clean)),
        "dates":         ", ".join(extract_dates(s_clean))
    })

df_narrative = pd.DataFrame(narrative_rows)
print(f"Narrative sentences extracted: {len(df_narrative)} rows.")


# ------------------------------------------------------------------------
# PART 7 — TABLE-LIKE STRUCTURE PARSING FROM LINE-LEVEL OCR
# ------------------------------------------------------------------------
# We want to interpret groups of numeric lines as potential tables. We:
#   1) Discover blocks of consecutive numeric lines.
#   2) Try to find year headers (e.g. "2024 2023") above or in the block.
#   3) For each line:
#         * determine the label
#         * assign numbers to year columns (year_2024, year_2023, ...)
#   4) Store results in df_tables.

print("\nParsing table-like blocks into structured rows...")

table_entries = []

for page_index, lines in enumerate(all_lines_by_page, start=1):
    if not lines:
        continue

    # Flag lines containing numeric tokens
    numeric_flag = [1 if extract_financial_numbers(l[0]) else 0 for l in lines]

    i = 0
    while i < len(lines):
        if numeric_flag[i] == 1:
            # Start of a numeric block
            start = i
            j = i + 1

            # Extend the block while lines are numeric or very close vertically
            while j < len(lines) and (numeric_flag[j] == 1 or j - start < 6):
                j += 1

            block = lines[start:j]

            # If no numeric lines in block, skip (safety check)
            if sum(numeric_flag[start:j]) < 1:
                i = j
                continue

            # Try to detect header years in top lines of block
            header_text = ""
            header_years = []

            for cand in block[:3]:
                yrs = extract_years(cand[0])
                if yrs:
                    header_text = cand[0]
                    header_years = yrs
                    break

            # If not found, look above the block in previous lines on same page
            if not header_years and start - 1 >= 0:
                for k in range(max(0, start - 2), start):
                    yrs = extract_years(lines[k][0])
                    if yrs:
                        header_text = lines[k][0]
                        header_years = yrs
                        break

            # Process each row in the block
            for row_text, left, top, width, height in block:
                raw = row_text.strip()
                nums = [v for _, v in extract_financial_numbers(raw)]

                # Find where the first money-like token occurs to split label/value
                first_money = None
                for m in re.finditer(_money_re, raw):
                    first_money = m
                    break

                if first_money:
                    label = raw[:first_money.start()].strip(" -:;,")
                else:
                    # Fallback: take first chunk before big gap
                    label = re.split(r"\s{2,}", raw)[0].strip()

                entry = {
                    "page":      page_index,
                    "section":   classify_section(header_text if header_text else label),
                    "line_item": sanitize_label(label),
                    "names":     ", ".join(extract_names(raw)),
                    "dates":     ", ".join(extract_dates(raw)),
                }

                # If we have header years, map numbers to those exact years
                if header_years:
                    for idx_y, y in enumerate(header_years):
                        entry[f"year_{y}"] = nums[idx_y] if idx_y < len(nums) else ""

                    # 'current' = latest year, 'prior' = previous year if available
                    sorted_years = sorted(header_years, reverse=True)
                    entry["value_current"] = entry.get(f"year_{sorted_years[0]}", "")
                    entry["value_prior"]   = entry.get(f"year_{sorted_years[1]}", "") if len(sorted_years) > 1 else ""
                else:
                    # No explicit year headers; just assign sequentially
                    for idx_n, val in enumerate(nums):
                        entry[f"year_{idx_n+1}"] = val
                    entry["value_current"] = nums[0] if len(nums) > 0 else ""
                    entry["value_prior"]   = nums[1] if len(nums) > 1 else ""

                table_entries.append(entry)

            i = j  # skip to end of this block
        else:
            i += 1

df_tables = pd.DataFrame(table_entries).fillna("")

# Deduplicate rows by (section, line_item), because OCR can sometimes
# produce duplicate lines.
if not df_tables.empty:
    df_tables["line_item"] = (
        df_tables["line_item"]
        .astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

    seen_keys = set()
    unique_rows = []
    for _, row in df_tables.iterrows():
        key = (row.get("section", ""), row.get("line_item", ""))
        if key in seen_keys:
            continue
        seen_keys.add(key)
        unique_rows.append(row.to_dict())

    df_tables = pd.DataFrame(unique_rows)

print(f"Structured table rows detected: {len(df_tables)}")


# ------------------------------------------------------------------------
# PART 8 — METRIC EXTRACTION & YEAR-ON-YEAR (YOY) ANALYSIS
# ------------------------------------------------------------------------

print("\nComputing key metrics and year-on-year comparisons...\n")


def find_metric(keywords):
    """
    Try to extract a financial metric by searching for one or more
    keywords in:
      1) df_tables["line_item"] first (structured)
      2) df_narrative["text"] if not found in tables

    Returns:
      A numeric value (float-like) or "" if nothing found.
    """

    # Search structured table rows first
    for kw in keywords:
        mask = df_tables["line_item"].str.lower().str.contains(kw, na=False)
        if mask.any():
            row = df_tables[mask].iloc[0]

            # Prefer the highest year_XXXX column if present
            year_cols = sorted(
                [c for c in row.keys() if c.startswith("year_")],
                reverse=True
            )
            if year_cols:
                return row[year_cols[0]]

            return row.get("value_current", "")

    # Fallback to narrative sentences
    for kw in keywords:
        mask = df_narrative["text"].str.lower().str.contains(kw, na=False)
        if mask.any():
            row = df_narrative[mask].iloc[0]
            if row["value_current"]:
                return row["value_current"]

    return ""


# Build dictionary of metrics with human-readable keys
metrics = {}
metrics["Turnover"]              = find_metric(["turnover", "sales"])
metrics["Profit for the year"]   = find_metric(["profit for the financial year", "profit for the year", "profit"])
metrics["Tax"]                   = find_metric(["tax on profit", "corporation tax", "tax"])
metrics["Tangible assets"]       = find_metric(["tangible assets", "tangible"])
metrics["Stocks"]                = find_metric(["stocks", "inventories"])
metrics["Debtors"]               = find_metric(["debtors", "trade debtors"])
metrics["Cash"]                  = find_metric(["cash at bank", "cash at bank and in hand", "cash"])
metrics["Creditors <1 year"]     = find_metric(["falling due within one year", "within one year"])
metrics["Creditors >1 year"]     = find_metric(["falling due after more than one year", "after more than"])
metrics["Net current assets"]    = find_metric(["net current assets"])
metrics["Total assets less current liabs"] = find_metric(["total assets less current liabilities"])
metrics["Net assets"]            = find_metric(["net assets"])
metrics["Share capital"]         = find_metric(["called up share capital", "share capital"])


# Extract employees from narrative text (if mentioned)
employees = ""
for txt in df_narrative["text"]:
    m = re.search(r"employees?.{0,30}?(\d{1,4})", txt, flags=re.IGNORECASE)
    if m:
        employees = m.group(1)
        break


# ------------------ YOY (Year-on-Year) Summary ------------------

yoy_summary = {}   # metric_name -> % change

# Identify all year_* columns present in df_tables
all_year_cols = sorted({col for col in df_tables.columns if col.startswith("year_")})
years = [col.replace("year_", "") for col in all_year_cols]

if len(years) >= 2:
    years = sorted(years)  # ascending order
    latest_year = years[-1]
    prev_year   = years[-2]

    for metric_name, val_current in metrics.items():

        # Only attempt YOY for metrics that exist numerically
        if val_current in ("", None):
            continue

        # We attempt to re-locate the metric row for older year data
        if metric_name == "Turnover":
            keywords = ["turnover", "sales"]
        elif metric_name == "Profit for the year":
            keywords = ["profit for the financial year", "profit for the year", "profit"]
        else:
            keywords = [metric_name.lower()]

        older_val = None

        for kw in keywords:
            mask = df_tables["line_item"].str.lower().str.contains(kw, na=False)
            if mask.any():
                row = df_tables[mask].iloc[0]
                older_col = f"year_{prev_year}"
                if older_col in row and row[older_col] not in ("", None):
                    older_val = row[older_col]
                break

        try:
            if older_val not in ("", None):
                v_new = float(val_current)
                v_old = float(older_val)
                if v_old != 0:
                    change_pct = ((v_new - v_old) / v_old) * 100.0
                    yoy_summary[metric_name] = round(change_pct, 2)
        except:
            pass


# ------------------------------------------------------------------------
# PART 8B — HUMAN-READABLE SUMMARY IN THE COLAB OUTPUT
# ------------------------------------------------------------------------
# This prints a friendly summary you can read directly in the notebook
# without opening Excel or the generated PDF.

print("\n======================")
print("         SUMMARY")
print("======================\n")


def fmt(value):
    """Format a numeric value as €x,xxx or return 'N/A'."""
    if value in ("", None):
        return "N/A"
    try:
        return f"€{float(value):,.0f}"
    except:
        return str(value)


# 1. Key metrics
print("🔹 KEY FINANCIAL METRICS:")
for metric, value in metrics.items():
    print(f"   • {metric:30s}: {fmt(value)}")

print(f"   • {'Employees':30s}: {employees if employees else 'N/A'}")


# 2. YOY changes
print("\n🔹 YEAR-ON-YEAR CHANGES (if two years detected):")
if yoy_summary:
    for metric, pct in yoy_summary.items():
        direction = "📈 Increase" if pct > 0 else "📉 Decrease"
        print(f"   • {metric:30s}: {pct}% ({direction})")
else:
    print("   Only one year detected or insufficient data for YOY.")


# 3. Directors' report highlights
print("\n🔹 DIRECTORS’ REPORT HIGHLIGHTS:")
directors_highlights = df_narrative[df_narrative["section"] == "directors_report"]["text"].tolist()
if not directors_highlights:
    print("   No directors' report text detected.")
else:
    for point in directors_highlights[:5]:
        snippet = point[:250] + ("..." if len(point) > 250 else "")
        print(f"   • {snippet}")


# 4. Notes to the accounts
print("\n🔹 NOTES TO THE ACCOUNTS (Key Policies):")
notes_highlights = df_narrative[df_narrative["section"] == "notes"]["text"].tolist()
if not notes_highlights:
    print("   No notes text detected.")
else:
    for point in notes_highlights[:5]:
        snippet = point[:250] + ("..." if len(point) > 250 else "")
        print(f"   • {snippet}")


# 5. Assets & liabilities overview
print("\n🔹 ASSET & LIABILITY OVERVIEW:")
print(f"   • Tangible Assets:            {fmt(metrics.get('Tangible assets'))}")
print(f"   • Stocks:                     {fmt(metrics.get('Stocks'))}")
print(f"   • Debtors:                    {fmt(metrics.get('Debtors'))}")
print(f"   • Cash:                       {fmt(metrics.get('Cash'))}")
print(f"   • Creditors <1 Year:          {fmt(metrics.get('Creditors <1 year'))}")
print(f"   • Creditors >1 Year:          {fmt(metrics.get('Creditors >1 year'))}")
print(f"   • Net Current Assets:         {fmt(metrics.get('Net current assets'))}")
print(f"   • Net Assets:                 {fmt(metrics.get('Net assets'))}")


# 6. Summary sentence on performance
print("\n🔹 PERFORMANCE SUMMARY:")
turn = metrics.get("Turnover")
prof = metrics.get("Profit for the year")
if turn and prof:
    print(f"   The company recorded turnover of {fmt(turn)} and a profit of {fmt(prof)} "
          f"in the latest financial year.")
else:
    print("   Turnover and/or profit figures could not be reliably detected.")

print("\n======================")
print(" END OF SUMMARY REPORT")
print("======================\n")


# ------------------------------------------------------------------------
# PART 9 — BUILD CHARTS (TURNOVER & PROFIT BY YEAR)
# ------------------------------------------------------------------------

print("Building charts for Turnover and Profit (if data series found)...")

chart_images = []  # will store (title, BytesIO_png_image)


def build_series_from_tables(item_keywords):
    """
    For a given set of keywords, try to find a matching line_item in df_tables
    and build a year → value dictionary from its year_* columns.

    Returns: (years_sorted_list, values_list) or (None, None) if not found.
    """
    for kw in item_keywords:
        mask = df_tables["line_item"].str.lower

# Continue chart builder function
        mask = df_tables["line_item"].str.lower().str.contains(kw, na=False)
        if mask.any():
            row = df_tables[mask].iloc[0]

            # Extract all year columns for this metric
            year_data = {}
            for col in row.index:
                if col.startswith("year_") and row[col] not in ("", None):
                    year = col.replace("year_", "")
                    try:
                        year_data[year] = float(row[col])
                    except:
                        pass

            if year_data:
                years_sorted = sorted(year_data.keys())      # ascending
                values_sorted = [year_data[y] for y in years_sorted]
                return years_sorted, values_sorted

    return None, None


# Build Turnover chart
turn_years, turn_vals = build_series_from_tables(["turnover", "sales"])
if turn_years:
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(turn_years, turn_vals, marker="o")
    ax.set_title("Turnover by Year")
    ax.set_xlabel("Year")
    ax.set_ylabel("Turnover (€)")
    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)

    chart_images.append(("Turnover by Year", buf))


# Build Profit chart
prof_years, prof_vals = build_series_from_tables(["profit for the financial year", "profit"])
if prof_years:
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(prof_years, prof_vals, marker="o", color="green")
    ax.set_title("Profit by Year")
    ax.set_xlabel("Year")
    ax.set_ylabel("Profit (€)")
    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)

    chart_images.append(("Profit by Year", buf))


print("Charts built:", len(chart_images))


# ------------------------------------------------------------------------
# PART 10 — EXPORT TO EXCEL
# ------------------------------------------------------------------------

excel_name = "financial_output.xlsx"
with pd.ExcelWriter(excel_name, engine="openpyxl") as writer:
    df_narrative.to_excel(writer, sheet_name="Narrative", index=False)
    df_tables.to_excel(writer, sheet_name="Tables", index=False)

    # Metrics summary sheet
    pd.DataFrame(
        list(metrics.items()),
        columns=["Metric", "Value"]
    ).to_excel(writer, sheet_name="Summary", index=False)

print("Excel generated:", excel_name)
files.download(excel_name)


# ------------------------------------------------------------------------
# PART 11 — BUILD STYLED PDF REPORT
# ------------------------------------------------------------------------

pdf_name = "Financial_Summary_Report.pdf"
styles = getSampleStyleSheet()
styleH = ParagraphStyle("Heading", parent=styles["Heading1"], alignment=1)
styleN = styles["Normal"]
styleB = ParagraphStyle("Bold", parent=styles["Normal"], fontName="Helvetica-Bold")

story = []
doc = SimpleDocTemplate(pdf_name, pagesize=A4, rightMargin=36, leftMargin=36, topMargin=36, bottomMargin=36)

# Title
story.append(Paragraph("Financial Summary Report", styleH))
story.append(Paragraph(f"Source Document: {pdf_path}", styleN))
story.append(Spacer(1, 0.2 * inch))

# Metrics table
table_data = [["Metric", "Value"]]
for k, v in metrics.items():
    table_data.append([k, fmt(v)])

tbl = RLTable(table_data, colWidths=[2.5 * inch, 3.5 * inch])
tbl.setStyle(TableStyle([
    ("BACKGROUND", (0, 0), (-1, 0), colors.lightgrey),
    ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
    ("GRID", (0, 0), (-1, -1), 0.25, colors.black),
]))
story.append(tbl)
story.append(Spacer(1, 0.3 * inch))

# Directors' highlights
story.append(Paragraph("Directors' Report Highlights", styleB))
if directors_highlights:
    for t in directors_highlights[:5]:
        story.append(Paragraph("- " + t[:300] + ("..." if len(t) > 300 else ""), styleN))
else:
    story.append(Paragraph("No directors' report content detected.", styleN))

story.append(Spacer(1, 0.3 * inch))

# Notes highlights
story.append(Paragraph("Notes to the Accounts (Extracts)", styleB))
if notes_highlights:
    for t in notes_highlights[:5]:
        story.append(Paragraph("- " + t[:300] + ("..." if len(t) > 300 else ""), styleN))
else:
    story.append(Paragraph("No notes extracted.", styleN))

story.append(PageBreak())

# Charts
if chart_images:
    story.append(Paragraph("Charts", styleH))
    for title, img_buf in chart_images:
        story.append(Paragraph(title, styleB))
        story.append(RLImage(img_buf, width=6.0 * inch, height=2.5 * inch))
        story.append(Spacer(1, 0.3 * inch))
else:
    story.append(Paragraph("No charts available.", styleN))

# Build PDF
doc.build(story)
print("PDF generated:", pdf_name)
files.download(pdf_name)

print("\n\n🎉 All tasks complete — Excel + PDF have been downloaded.")



'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: C:\Users\oisin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google'